In [9]:
!docker-compose up -d db

!docker-compose ps

[+] Running 1/2
 ✔ Network imdb-project_default  Created                                   0.0s 
 ⠋ Container imdb-db             Starting                                  0.1s 
[+] Running 2/2
 ✔ Network imdb-project_default  Created                                   0.0s 
 ✔ Container imdb-db             Started                                   0.1s 
NAME      IMAGE         COMMAND                  SERVICE   CREATED        STATUS                                     PORTS
imdb-db   postgres:18   "docker-entrypoint.s…"   db        1 second ago   Up Less than a second (health: starting)   0.0.0.0:5432->5432/tcp, [::]:5432->5432/tcp


In [2]:
import pandas as pd
import gzip
import requests
import shutil
import os
from sys import path
from sqlalchemy import create_engine
from dotenv import load_dotenv
import logging

logging.basicConfig(level=logging.INFO,format='%(asctime)s - %(levelname)s - %(message)s')

path.append(os.path.realpath("../"))
load_dotenv()

NAME_BASIC_URL = "https://datasets.imdbws.com/name.basics.tsv.gz"

NAMES_ZIP_FILE = "name.basics.tsv.gz"
NAMES_FILE = "name.basics.tsv"

if not os.path.isfile(NAMES_FILE):
    response = requests.get(NAME_BASIC_URL, stream=True)
    with open(NAMES_ZIP_FILE, "wb") as f:
        shutil.copyfileobj(response.raw, f)

    with gzip.open("name.basics.tsv.gz", "rb") as f_in:
        with open("name.basics.tsv", "wb") as f_out:
            shutil.copyfileobj(f_in, f_out)

df_actors = pd.read_csv("name.basics.tsv", sep="\t")


selected_columns = [
    "nconst",
    "primaryName",
    "birthYear",
    "primaryProfession"
]
clean_df = df_actors[selected_columns]
engine = create_engine(os.getenv("DATABASE_URL"))
try:
    table_name = "actors"
    clean_df.to_sql(table_name,engine, if_exists="replace", index=False)
    table = pd.read_sql("SELECT * FROM actors LIMIT 10", engine)
    logging.info(f"\n{table}") 
except Exception as e:
    logging.error("Exception:", e)
finally:
    engine.dispose()

--- Logging error ---
Traceback (most recent call last):
  File "/Users/miguelcosano/imdb-project/.venv/lib/python3.13/site-packages/sqlalchemy/engine/base.py", line 143, in __init__
    self._dbapi_connection = engine.raw_connection()
                             ~~~~~~~~~~~~~~~~~~~~~^^
  File "/Users/miguelcosano/imdb-project/.venv/lib/python3.13/site-packages/sqlalchemy/engine/base.py", line 3301, in raw_connection
    return self.pool.connect()
           ~~~~~~~~~~~~~~~~~^^
  File "/Users/miguelcosano/imdb-project/.venv/lib/python3.13/site-packages/sqlalchemy/pool/base.py", line 447, in connect
    return _ConnectionFairy._checkout(self)
           ~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^
  File "/Users/miguelcosano/imdb-project/.venv/lib/python3.13/site-packages/sqlalchemy/pool/base.py", line 1264, in _checkout
    fairy = _ConnectionRecord.checkout(pool)
  File "/Users/miguelcosano/imdb-project/.venv/lib/python3.13/site-packages/sqlalchemy/pool/base.py", line 711, in checkout
    rec = 

In [10]:
import logging
import os
import sys
import gzip
import shutil
import requests
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
from io import StringIO
import csv

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

sys.path.append(os.path.realpath("../"))
load_dotenv()

NAME_BASIC_URL = "https://datasets.imdbws.com/name.basics.tsv.gz"
NAMES_ZIP_FILE = "name.basics.tsv.gz"
NAMES_FILE = "name.basics.tsv"

if not os.path.isfile(NAMES_FILE):
    logging.info("Downloading name.basics.tsv.gz...")
    response = requests.get(NAME_BASIC_URL, stream=True)
    with open(NAMES_ZIP_FILE, "wb") as f:
        shutil.copyfileobj(response.raw, f)
    
    logging.info("Extracting gzip file...")
    with gzip.open(NAMES_ZIP_FILE, "rb") as f_in:
        with open(NAMES_FILE, "wb") as f_out:
            shutil.copyfileobj(f_in, f_out)
    logging.info("Download complete")

def psql_insert_copy(table, conn, keys, data_iter):
    """
    Execute SQL statement inserting data

    Parameters
    ----------
    table : pandas.io.sql.SQLTable
    conn : sqlalchemy.engine.Engine or sqlalchemy.engine.Connection
    keys : list of str
        Column names
    data_iter : Iterable that iterates the values to be inserted
    """
    dbapi_conn = conn.connection
    with dbapi_conn.cursor() as cur:
        s_buf = StringIO()
        writer = csv.writer(s_buf)
        writer.writerows(data_iter)
        s_buf.seek(0)

        columns = ', '.join(['"{}"'.format(k) for k in keys])
        if table.schema:
            table_name = '{}.{}'.format(table.schema, table.name)
        else:
            table_name = table.name

        sql = 'COPY {} ({}) FROM STDIN WITH CSV'.format(
            table_name, columns)
        cur.copy_expert(sql=sql, file=s_buf)

engine = create_engine(os.getenv("DATABASE_URL"))
table_name = "actors"

try:
    logging.info("Reading TSV file in chunks...")
    selected_columns = ["nconst", "primaryName", "birthYear", "primaryProfession"]
    chunk_size = 100000
    
    for i, chunk in enumerate(pd.read_csv(
        NAMES_FILE, 
        sep="\t", 
        usecols=selected_columns,
        chunksize=chunk_size,
        na_values=['\\N'],
        keep_default_na=True,
        dtype={
        'nconst': str,
        'primaryName': str,
        'birthYear': 'Int64',
        'primaryProfession': str
    }
    )):
        chunk = chunk.where(pd.notnull(chunk), None)

        if_exists_mode = 'replace' if i == 0 else 'append'
        
        chunk.to_sql(
            table_name,
            engine,
            if_exists=if_exists_mode,
            index=False,
            method=psql_insert_copy
        )
        
        logging.info(f"Inserted chunk {i+1} ({len(chunk)} rows)")
    
    with engine.connect() as conn:
        result = pd.read_sql("SELECT COUNT(*) as total FROM actors", conn)
        logging.info(f"Total rows inserted: {result['total'][0]}")
    
except Exception as e:
    logging.error(f"Exception occurred: {e}")
    raise
finally:
    engine.dispose()
    logging.info("Database connection closed")

2025-12-07 14:37:16,306 - INFO - Reading TSV file in chunks...
2025-12-07 14:37:16,394 - ERROR - Exception occurred: (psycopg2.OperationalError) could not translate host name "db" to address: nodename nor servname provided, or not known

(Background on this error at: https://sqlalche.me/e/20/e3q8)
2025-12-07 14:37:16,395 - INFO - Database connection closed


OperationalError: (psycopg2.OperationalError) could not translate host name "db" to address: nodename nor servname provided, or not known

(Background on this error at: https://sqlalche.me/e/20/e3q8)

## Data insertion

After reviewing several options to insert the data into the postgressql database the best approach would be the copy one.

In [34]:
!docker-compose down -v

[+] Running 0/1
 ⠋ Container imdb-db  Stopping                                             0.1s 
[+] Running 0/1
 ⠙ Container imdb-db  Stopping                                             0.2s 
[+] Running 0/1
 ⠹ Container imdb-db  Stopping                                             0.3s 
[+] Running 1/3
 ✔ Container imdb-db                  Removed                              0.3s 
 ⠋ Volume imdb-project_postgres_data  Removing                             0.1s 
 ⠋ Network imdb-project_default       Re...                                0.1s 
[+] Running 2/3
 ✔ Container imdb-db                  Removed                              0.3s 
 ✔ Volume imdb-project_postgres_data  Removed                              0.1s 
 ⠙ Network imdb-project_default       Re...                                0.2s 
[+] Running 2/3
 ✔ Container imdb-db                  Removed                              0.3s 
 ✔ Volume imdb-project_postgres_data  Removed                              0.1s 
 ⠹ Network im